# TARDIS - Step 3: Building a Prediction Model

**Objectif**: Prédire la durée des retards de trains en minutes

## 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

## 2. Charger les données nettoyées

In [ ]:
# pour charger le fichier nettoyé 
df = pd.read_csv('cleaned_dataset.csv')

# convertir les colonnes numériques qui sont en string
cols_numeriques = [
    'Retard moyen de tous les trains à l\'arrivée',
    'Durée moyenne du trajet',
    'Nombre de circulations prévues',
    'Retard moyen des trains en retard à l\'arrivée',
]
for col in cols_numeriques:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.shape)
df.head()

In [ ]:
# et voir ce qu'il y a comme colonnes.
print(df.columns.tolist())

In [ ]:
# verifier s'il y a des valeurs manquantes
df.info()

## 3. Choisir la target et les features

In [ ]:
# la variable qu'on veut predire les retards à l'arrivée.
target = 'Retard moyen de tous les trains à l\'arrivée'

# les variables explicatives
colonnes_features = [
    'Service',
    'Gare de départ',
    'Gare d\'arrivée',
    'Durée moyenne du trajet',
    'Nombre de circulations prévues'
]

#  month/year 
if 'month' in df.columns:
    colonnes_features.append('month')
if 'year' in df.columns:
    colonnes_features.append('year')

In [ ]:
# enlever les lignes avec des NaN ( absence d'info).
donnees = df.dropna(subset=[target] + colonnes_features).copy()
print(f"Reste {len(donnees)} lignes après nettoyage")

In [ ]:
X = donnees[colonnes_features].copy()
y = donnees[target].copy()

print(X.shape, y.shape)

## explorer les retard

Juste pour voir à quoi ressemblent les retards

In [ ]:
print(y.describe())

In [ ]:
# histogramme simple
plt.figure(figsize=(9, 5))
plt.hist(y, bins=40)
plt.xlabel('Retard (minutes)')
plt.ylabel('Nombre de trajets')
plt.title('Distribution des retards')
plt.axvline(y.mean(), color='r', linestyle='--')
plt.show()

## Encodage des variables.

Les gares et le service sont en texte, il faut que je les transformer en chiffres

In [ ]:
# encoder service
le_service = LabelEncoder()
X['Service'] = le_service.fit_transform(X['Service'].astype(str))

# encoder gare depart
le_depart = LabelEncoder()
X['Gare de départ'] = le_depart.fit_transform(X['Gare de départ'].astype(str))

# encoder gare arrivee
le_arrivee = LabelEncoder()
X['Gare d\'arrivée'] = le_arrivee.fit_transform(X['Gare d\'arrivée'].astype(str))

In [ ]:
# vérifier que ça a marché
X.head()

## Train/Test

In [ ]:
# séparer en train (80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train:", X_train.shape)
print("Test:", X_test.shape)

## Baseline

On calcule la moyenne de y_train.
On prédit toujours cette moyenne pour y_test.

Et ainsi de suite pour calculer RMSE, R²..

In [ ]:
moyenne = y_train.mean()
print(f"Moyenne: {moyenne:.2f} minutes")

In [ ]:
# prédire toujours la moyenne
baseline_preds = [moyenne] * len(y_test)

mae_baseline = mean_absolute_error(y_test, baseline_preds)
rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_preds))
r2_baseline = r2_score(y_test, baseline_preds)

print(f"MAE baseline: {mae_baseline:.2f} min")
print(f"RMSE baseline: {rmse_baseline:.2f} min")
print(f"R²: {r2_baseline:.3f}")

## Test Linear Regression

In [ ]:
# créer et entrainjer le modèle.
modele_lineaire = LinearRegression()
modele_lineaire.fit(X_train, y_train)

In [ ]:
# prédictions
y_pred_linear = modele_lineaire.predict(X_test)

In [ ]:
# métriques
mae_lin = mean_absolute_error(y_test, y_pred_linear)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_linear))
r2_lin = r2_score(y_test, y_pred_linear)

print(f"MAE: {mae_lin:.2f} min")
print(f"RMSE: {rmse_lin:.2f} min")
print(f"R²: {r2_lin:.3f}")

print(f"\nC'est {((mae_baseline - mae_lin) / mae_baseline * 100):.1f}% mieux que le baseline")

## Test Random Forest

je vais essayer random forest, normalement ça devrait marcher.
voir si ce modèle fait mieux que la moyenne.

Random Forest = plusieurs arbres de décision.


In [ ]:
# j'ai mis max_depth=10 mais, faudra quej j'optimise ça plus tard.
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
predictions = rf_model.predict(X_test)

In [ ]:
mae_rf = mean_absolute_error(y_test, predictions)
rmse_rf = np.sqrt(mean_squared_error(y_test, predictions))
r2_rf = r2_score(y_test, predictions)

print(f"MAE: {mae_rf:.2f} min")
print(f"RMSE: {rmse_rf:.2f} min")
print(f"R²: {r2_rf:.3f}")

print(f"\nAmélioration vs baseline: {((mae_baseline - mae_rf) / mae_baseline * 100):.1f}%")

## Comparaison

In [ ]:
# récap du tableau.
comparaison = pd.DataFrame({
    'Modèle': ['Baseline', 'Linear Reg', 'Random Forest'],
    'MAE': [mae_baseline, mae_lin, mae_rf],
    'RMSE': [rmse_baseline, rmse_lin, rmse_rf],
    'R²': [r2_baseline, r2_lin, r2_rf]
})

print(comparaison)

In [ ]:
# graphique mais ça donne une idée assez globale on peut dire..
fig, axes = plt.subplots(1, 3, figsize=(13, 4))

axes[0].bar(comparaison['Modèle'], comparaison['MAE'])
axes[0].set_title('MAE (minutes)')
axes[0].tick_params(axis='x', rotation=30)

axes[1].bar(comparaison['Modèle'], comparaison['RMSE'])
axes[1].set_title('RMSE (minutes)')
axes[1].tick_params(axis='x', rotation=30)

axes[2].bar(comparaison['Modèle'], comparaison['R²'])
axes[2].set_title('R²')
axes[2].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

## Importance des features

Pour voir quelles variables sont les plus importantes pour random forest

In [ ]:
importances = rf_model.feature_importances_

df_imp = pd.DataFrame({
    'Feature': colonnes_features,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print(df_imp)

In [ ]:
plt.figure(figsize=(9, 5))
plt.barh(df_imp['Feature'], df_imp['Importance'])
plt.xlabel('Importance')
plt.title('Features importantes')
plt.tight_layout()
plt.show()

## Visualisation des prédictions

In [ ]:
# scatter pour voir si les predictions collent à la realite, voir si le modèle prédit bien visuellement.

plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Retard réel (min)')
plt.ylabel('Retard prédit (min)')
plt.title('Prédictions vs Réalité')
plt.grid(True, alpha=0.2)
plt.show()

## Sauvegarde du modèle

In [ ]:
# random forest marche mieux donc je sauvegarde celui-là.
if mae_rf < mae_lin:
    final_model = rf_model
    model_name = 'Random Forest'
else:
    final_model = modele_lineaire
    model_name = 'Linear Regression'

print(f"Modèle choisi: {model_name}")

In [ ]:
# sauvegarder le modele
joblib.dump(final_model, 'model.joblib')

# sauvegarder les encoders aussi
encoders_dict = {
    'Service': le_service,
    'Gare de départ': le_depart,
    'Gare d\'arrivée': le_arrivee
}
joblib.dump(encoders_dict, 'encoders.joblib')

# et les noms de features
joblib.dump(colonnes_features, 'features.joblib')

print("Sauvegardé: model.joblib, encoders.joblib, features.joblib")

## Test

In [ ]:
# test sur des exemples
test_samples = X_test[:5]
test_real = y_test[:5]
test_preds = final_model.predict(test_samples)

print("Exemples:\n")
for i in range(5):
    real = test_real.iloc[i]
    pred = test_preds[i]
    diff = abs(real - pred)
    print(f"{i+1}. Réel: {real:.1f} min | Prédit: {pred:.1f} min | Diff: {diff:.1f} min")

## Bilan

### Ce qu'on a fait
On a testé 2 modèles pour prédire les retards : une régression linéaire et un random forest. Le random forest marche mieux.

### Résultats
Le modèle prédit les retards avec une erreur de quelques minutes en moyenne. C'est pas parfait mais c'est quand même bien mieux que si on prédisait juste la moyenne tout le temps.

Les gares et la durée du trajet ont l'air d'être ce qui influe le plus sur les retards.

### Problèmes
- Le modèle se plante un peu sur les très gros retards (plus de 30 min)
- Les trucs exceptionnels comme les grèves ou la météo sont pas bien pris en compte
- Y'a des trajets qu'on a très peu dans les données

### Ce qu'on pourrait améliorer
- Essayer d'autres algorithmes
- Mieux régler les paramètres du modèle
- Rajouter des infos comme la météo ou si c'est un jour férié
- Peut-être traiter les retards extrêmes à part